In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os

In [16]:
snr_cat = pd.read_pickle('../dataframes/high_snr_df.pkl')

In [17]:
idx = 1
for i in snr_cat.index:
    slrm_text= ["#!/bin/bash\n#SBATCH -J bfit-{0}\n#SBATCH -p background\n#SBATCH --time=8:00:00\n#SBATCH --mem-per-cpu=32000\n#SBATCH -o bfit{0}-%j.out\n\
#SBATCH -e bfit{0}-%j.err\n\nchmod a+x bestfit.py\n\n\
python bestfit.py {1} {2} {3} &\n\
\nwait\n\nexit 0".format(idx, snr_cat.field[i],snr_cat.id[i], np.round(snr_cat.z[i],4))]
    np.savetxt('../cluster_scripts/bestfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
    idx +=1


In [12]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J bfit-61
#SBATCH -p background
#SBATCH --time=2:00:00
#SBATCH --mem-per-cpu=16000
#SBATCH -o bfit61-%j.out
#SBATCH -e bfit61-%j.err

chmod a+x bestfit.py

python bestfit.py GSD 35774 1.2533 &

wait

exit 0


In [2]:
select = pd.read_pickle('../spec_files/all_section.pkl')
select = select.query('AGN != "AGN" and use == True')

In [11]:
idx = 1
test_ids = [21156, 39170, 41520]
for i in select.index:   
    if select.id[i] in test_ids:
        if os.path.isfile('../data/posteriors/{0}_{1}_zfit_Pz.npy'.format(select.field[i], select.id[i])):
            z,pz = np.load('../data/posteriors/{0}_{1}_zfit_Pz.npy'.format(select.field[i], select.id[i]))
            zfit = z[pz == max(pz)][0] 
        else:
            zfit =  select.zphot[i]

        slrm_text= ["#!/bin/bash\n#SBATCH -J ttst-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o ttst{0}-%j.out\n#SBATCH -e ttst{0}-%j.err\n\nchmod a+x fullfit-test.py\n\n\
python fullfit-test.py {1} {2} {3:1.4f}\n\
\nwait\n\nexit 0".format(idx, select.field[i], select.id[i], zfit)]
        np.savetxt('../cluster_scripts/fullfit-test_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
        idx +=1   
        

In [10]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J ttst-3
#SBATCH -p background
#SBATCH -c 8
#SBATCH --time=16:00:00
#SBATCH --mem=32gb
#SBATCH -o ttst3-%j.out
#SBATCH -e ttst3-%j.err

chmod a+x fullfit-test.py

python fullfit-test.py GSD 41520 1.6148

wait

exit 0


In [3]:
idx = 1
for i in select.index:   
    if select.use[i]:
        slrm_text= ["#!/bin/bash\n#SBATCH -J zfit-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o zfit{0}-%j.out\n#SBATCH -e zfit{0}-%j.err\n\nchmod a+x redshift_fit.py\n\n\
python redshift_fit.py {1} {2} {3:1.4f}\n\
\nwait\n\nexit 0".format(idx, select.field[i], select.id[i], select.zphot[i])]
        np.savetxt('../cluster_scripts/zfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
        idx +=1                  

In [4]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J zfit-166
#SBATCH -p background
#SBATCH -c 8
#SBATCH --time=16:00:00
#SBATCH --mem=32gb
#SBATCH -o zfit166-%j.out
#SBATCH -e zfit166-%j.err

chmod a+x redshift_fit.py

python redshift_fit.py GSD 49072 0.9914

wait

exit 0


In [23]:
idx = 1

errterm = [0.02,0.03,0.04,0.05]
for i in errterm:
    for ii in range(10):   
        slrm_text= ["#!/bin/bash\n#SBATCH -J tfit-{1}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o tfit{1}-%j.out\n#SBATCH -e tfit{1}-%j.err\n\nchmod a+x fullfit-test.py\n\n\
python fullfit-test.py GND 21156 1.255 {0} {1}\n\
\nwait\n\nexit 0".format(i, idx)]
        np.savetxt('../cluster_scripts/fullfit-test_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
        idx +=1  

In [2]:
flist = glob('../data/posteriors/*zfit*')

fld = [os.path.basename(U).split('_')[0] for U in flist] 
ids = [os.path.basename(U).split('_')[1] for U in flist] 

In [13]:
idx = 1
for i in select.index:   
    if os.path.isfile('../data/posteriors/{0}_{1}_zfit_Pz.npy'.format(select.field[i], select.id[i])):
        z,pz = np.load('../data/posteriors/{0}_{1}_zfit_Pz.npy'.format(select.field[i], select.id[i]))
        zfit = z[pz == max(pz)][0] 
    else:
        zfit =  select.zphot[i]
        
    slrm_text= ["#!/bin/bash\n#SBATCH -J tfit-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o tfit{0}-%j.out\n#SBATCH -e tfit{0}-%j.err\n\nchmod a+x fullfit.py\n\n\
python fullfit.py {1} {2} {3:1.4f}\n\
\nwait\n\nexit 0".format(idx, select.field[i], select.id[i], zfit)]
    np.savetxt('../cluster_scripts/fullfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
    idx +=1  

In [28]:
idx = 1
for i in select.index: 
    if os.path.isfile('../data/posteriors/{0}_{1}_zfit_Pz.npy'.format(select.field[i], select.id[i]))
        z,pz = np.load('../data/posteriors/{0}_{1}_zfit_Pz.npy'.format(select.field[i], select.id[i]))
        zfit = z[pz == max(pz)][0] 
    else:
        zfit =  select.zphot[i]
    slrm_text= ["#!/bin/bash\n#SBATCH -J dfit-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o dfit{0}-%j.out\n#SBATCH -e dfit{0}-%j.err\n\nchmod a+x delayfit.py\n\n\
python delayfit.py {1} {2} {3:1.4f}\n\
\nwait\n\nexit 0".format(idx, select.field[i], select.id[i], zfit)]
    np.savetxt('../cluster_scripts/delayfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
    idx +=1  

In [12]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J tfit-166
#SBATCH -p background
#SBATCH -c 8
#SBATCH --time=16:00:00
#SBATCH --mem=32gb
#SBATCH -o tfit166-%j.out
#SBATCH -e tfit166-%j.err

chmod a+x fullfit.py

python fullfit.py GSD 49072 1.0630

wait

exit 0


In [13]:
####ALMA Zfit########
gals = [26272, 29257, 38843, 39012, 39364, 40223, 41520, 42113, 42778,
        43007, 44042]

LIMSR = [None, None, None, None, None, [11200,12500], None, [11200,15400], [14000,16500], None, None]

idx = 1

for i in range(len(gals)):
    if LIMSR[i] == None:
        lim1 = 11200
        lim2 = 16500
    else:
        lim1 = LIMSR[i][0]
        lim2 = LIMSR[i][1]
    
    slrm_text= ["#!/bin/bash\n#SBATCH -J afit-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o afit{0}-%j.out\n#SBATCH -e afit{0}-%j.err\n\nchmod a+x ALMA_Zfit.py\n\n\
python ALMA_Zfit.py {1} {2} {3}\n\
\nwait\n\nexit 0".format(idx, gals[i], lim1, lim2)]
    np.savetxt('../cluster_scripts/ALMA_Zfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
    idx +=1  

In [12]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J afit-11
#SBATCH -p background
#SBATCH -c 8
#SBATCH --time=16:00:00
#SBATCH --mem=32gb
#SBATCH -o afit11-%j.out
#SBATCH -e afit11-%j.err

chmod a+x ALMA_Zfit.py

python ALMA_Zfit.py 44042 11200 16500

wait

exit 0


In [2]:
########SF fit###########
SF_db = pd.read_pickle('../Casey_data/SF_db.pkl')

In [3]:
idx = 1
for i in SF_db.index:      
    slrm_text= ["#!/bin/bash\n#SBATCH -J sfit-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o sfit{0}-%j.out\n#SBATCH -e sfit{0}-%j.err\n\nchmod a+x SFfit-test.py\n\n\
python SFfit-test.py {1} {2} {3:1.5f} {4}\n\
\nwait\n\nexit 0".format(idx, SF_db.field[i], SF_db.id[i], SF_db.zgrism[i], SF_db.lmass[i])]
    np.savetxt('../cluster_scripts/SFfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
    idx +=1  

In [4]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J sfit-69
#SBATCH -p background
#SBATCH -c 8
#SBATCH --time=16:00:00
#SBATCH --mem=32gb
#SBATCH -o sfit69-%j.out
#SBATCH -e sfit69-%j.err

chmod a+x SFfit-test.py

python SFfit-test.py GSD 40078 1.29567 9.556964641311804

wait

exit 0


In [2]:
SF_db = pd.read_pickle('../Casey_data/SF_db_p1.pkl')

In [3]:
idx = 1
for i in SF_db.index:  
    
    x,px = np.load('../Casey_data/posteriors/{0}_{1}_SFfit_p1_Pa.npy'.format(SF_db.field[i], SF_db.id[i]))
    a = x[px == max(px)][0]
    
    slrm_text= ["#!/bin/bash\n#SBATCH -J sfit-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o sfit{0}-%j.out\n#SBATCH -e sfit{0}-%j.err\n\nchmod a+x SFfit-sim.py\n\n\
python SFfit-sim.py {1} {2} {3:1.5f} {4:1.3f} {5:1.3f} {6:1.3f} {7:1.3f} {8:1.3f} {9:1.3f} {10:1.3f} {11:1.3f} {12:1.3f} {13:1.3f}\n\
\nwait\n\nexit 0".format(idx, SF_db.field[i], SF_db.id[i], SF_db.zgrism[i], SF_db.lmass[i], 
            SF_db.Av[i], SF_db.Z[i], a, SF_db.m1[i], SF_db.m2[i], SF_db.m3[i], SF_db.m4[i], SF_db.m5[i], SF_db.m6[i])]
    np.savetxt('../cluster_scripts/SFfit_sim_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
    idx +=1  

In [4]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J sfit-69
#SBATCH -p background
#SBATCH -c 8
#SBATCH --time=16:00:00
#SBATCH --mem=32gb
#SBATCH -o sfit69-%j.out
#SBATCH -e sfit69-%j.err

chmod a+x SFfit-sim.py

python SFfit-sim.py GSD 40078 1.29567 9.608 0.632 0.090 2.797 0.055 0.055 0.115 0.278 0.336 0.080

wait

exit 0


In [14]:
idx = 1
fields = ['GSD', 'GND']


for i in range(len(fields)):
    if fields[i] == 'GSD':
        subfields = ['ERSPRIME', 'GS1', 'GS2', 'GS3', 'GS4', 'GS5']
    else:
        subfields = ['GN1', 'GN2', 'GN3', 'GN4', 'GN5', 'GN7']
    
    for ii in range(len(subfields)):
        slrm_text= ["#!/bin/bash\n#SBATCH -J bex-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=64gb\n#SBATCH -o bex{0}-%j.out\n#SBATCH -e bex{0}-%j.err\n\nchmod a+x beam_extract.py\n\n\
python beam_extract.py {1} {2} \n\
\nwait\n\nexit 0".format(idx, fields[i], subfields[ii])]
        np.savetxt('../cluster_scripts/beamextract_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
        idx +=1

In [15]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J bex-12
#SBATCH -p background
#SBATCH -c 8
#SBATCH --time=16:00:00
#SBATCH --mem=64gb
#SBATCH -o bex12-%j.out
#SBATCH -e bex12-%j.err

chmod a+x beam_extract.py

python beam_extract.py GND GN7 

wait

exit 0


# Buldge quiescent

In [2]:
mdb = pd.read_pickle('../dataframes/fitdb/fullfitdb.pkl')
BSQ = pd.read_pickle('../dataframes/galaxy_frames/GS_buldge_Qup.pkl')
BNQ = pd.read_pickle('../dataframes/galaxy_frames/GN_buldge_Qup.pkl')

In [17]:
idx = 1
for i in BSQ.index:   
    if BSQ.id[i] not in mdb.id.values:
        zfit =  BSQ.zphot[i]

        slrm_text= ["#!/bin/bash\n#SBATCH -J bqf-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o bqf{0}-%j.out\n#SBATCH -e bqf{0}-%j.err\n\nchmod a+x ../fullfit.py\n\n\
python ../fullfit.py {1} {2} {3:1.4f}\n\
\nwait\n\nexit 0".format(idx, 'GSD', BSQ.id[i], zfit)]
        np.savetxt('../cluster_scripts/bq_fit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
        idx +=1  
        
for i in BNQ.index:   
    if BNQ.id[i] not in mdb.id.values:
        zfit =  BNQ.zphot[i]

        slrm_text= ["#!/bin/bash\n#SBATCH -J bqf-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o bqf{0}-%j.out\n#SBATCH -e bqf{0}-%j.err\n\nchmod a+x ../fullfit.py\n\n\
python ../fullfit.py {1} {2} {3:1.4f}\n\
\nwait\n\nexit 0".format(idx, 'GND', BNQ.id[i], zfit)]
        np.savetxt('../cluster_scripts/bq_fit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
        idx +=1  

In [18]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J bqf-91
#SBATCH -p background
#SBATCH -c 8
#SBATCH --time=16:00:00
#SBATCH --mem=32gb
#SBATCH -o bqf91-%j.out
#SBATCH -e bqf91-%j.err

chmod a+x ../fullfit.py

python ../fullfit.py GND 35090 1.1632

wait

exit 0


# Buldge star forming

In [29]:
sfdb = pd.read_pickle('../Casey_data/SF_db_p1.pkl')
BSSF = pd.read_pickle('../dataframes/galaxy_frames/GS_buldge_SFup.pkl')
BNSF = pd.read_pickle('../dataframes/galaxy_frames/GN_buldge_SFup.pkl')

In [30]:
idx = 1
for i in BSSF.index:   
    if BSSF.id[i] not in sfdb.id.values:
        slrm_text= ["#!/bin/bash\n#SBATCH -J bsf-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o bsf{0}-%j.out\n#SBATCH -e bsf{0}-%j.err\n\nchmod a+x ../SFfit.py\n\n\
python ../SFfit.py {1} {2} {3:1.5f} {4}\n\
\nwait\n\nexit 0".format(idx, 'GSD', BSSF.id[i], BSSF.z_50[i], BSSF.lmass[i])]
        np.savetxt('../cluster_scripts/bsf_fit_{0}.slrm'.format(idx), slrm_text, fmt='%s')    
        idx +=1  
        
for i in BNSF.index:   
    if BNSF.id[i] not in sfdb.id.values:
        slrm_text= ["#!/bin/bash\n#SBATCH -J bsf-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o bsf{0}-%j.out\n#SBATCH -e bsf{0}-%j.err\n\nchmod a+x ../SFfit.py\n\n\
python ../SFfit.py {1} {2} {3:1.5f} {4}\n\
\nwait\n\nexit 0".format(idx, 'GND', BNSF.id[i], BNSF.z_50[i], BNSF.lmass[i])]
        np.savetxt('../cluster_scripts/bsf_fit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
        idx +=1  

In [31]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J bsf-59
#SBATCH -p background
#SBATCH -c 8
#SBATCH --time=16:00:00
#SBATCH --mem=32gb
#SBATCH -o bsf59-%j.out
#SBATCH -e bsf59-%j.err

chmod a+x ../SFfit.py

python ../SFfit.py GND 38093 1.09406 10.019586308805085

wait

exit 0


In [2]:
alldb = pd.read_pickle('../dataframes/galaxy_frames/all_galaxies.pkl')

In [3]:
alldb

,field,id,zgrism,z_grizli,lmass,Qlines
0,GND,10639,NaN,1.265857,9.163849,[]
1,GND,11279,-99.000000,1.220616,10.471450,[]
2,GND,11460,-99.000000,0.557419,10.367308,[]
3,GND,11814,1.692009,NaN,10.393400,[]
4,GND,11839,NaN,1.240109,9.730826,[]
5,GND,12078,1.016371,NaN,10.797066,[]
6,GND,12451,1.010006,NaN,10.365516,[]
7,GND,12478,1.223559,NaN,10.392782,[]
8,GND,12481,0.663151,NaN,10.557659,[]
9,GND,12617,0.930914,NaN,9.835416,[]


In [4]:
idx = 1
for i in alldb.index:   
    if alldb.zgrism[i] > 0:
        
        ext = '{0} {1} {2:1.5f}'.format(alldb.field[i], alldb.id[i], alldb.zgrism[i])

        for ii in range(len(alldb.Qlines[i])):
            ext += ' {}'.format(alldb.Qlines[i][ii])
        
        slrm_text= ["#!/bin/bash\n#SBATCH -J mbq-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --mem=32gb\n#SBATCH -o mbq{0}-%j.out\n#SBATCH -e mbq{0}-%j.err\n\nchmod a+x ../fullfit-multifit.py\n\n\
python ../fullfit-multifit.py {1}\n\
\nwait\n\nexit 0".format(idx, ext)]
        np.savetxt('../cluster_scripts/MBfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')    
        idx +=1  
        
    else:
        slrm_text= ["#!/bin/bash\n#SBATCH -J mbsf-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --mem=32gb\n#SBATCH -o mbsf{0}-%j.out\n#SBATCH -e mbsf{0}-%j.err\n\nchmod a+x ../SFfit-multifit.py\n\n\
python ../SFfit-multifit.py {1} {2} {3:1.5f} {4}\n\
\nwait\n\nexit 0".format(idx, alldb.field[i], alldb.id[i], alldb.z_grizli[i], alldb.lmass[i])]
        np.savetxt('../cluster_scripts/MBfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')    
        idx +=1 

In [5]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J mbq-385
#SBATCH -p background
#SBATCH -c 8
#SBATCH --mem=32gb
#SBATCH -o mbq385-%j.out
#SBATCH -e mbq385-%j.err

chmod a+x ../fullfit-multifit.py

python ../fullfit-multifit.py GSD 49362 1.19791

wait

exit 0


In [5]:
idx = 1
for i in alldb.index:   
    if alldb.zgrism[i] > 0:
        
        ext = '{0} {1} {2:1.5f}'.format(alldb.field[i], alldb.id[i], alldb.zgrism[i])

        for ii in range(len(alldb.Qlines[i])):
            ext += ' {}'.format(alldb.Qlines[i][ii])
        
        slrm_text= ["#!/bin/bash\n#SBATCH -J mbx-{0}\n#SBATCH -p background\n#SBATCH -c 1\n\
#SBATCH --mem=8gb\n#SBATCH -o mbx{0}-%j.out\n#SBATCH -e mbx{0}-%j.err\n\nchmod a+x ../Extract_nuisance.py\n\n\
python ../Extract_nuisance.py {1}\n\
\nwait\n\nexit 0".format(idx, ext)]
        np.savetxt('../cluster_scripts/MBext_{0}.slrm'.format(idx), slrm_text, fmt='%s')    
        idx +=1  
        
    else:
        slrm_text= ["#!/bin/bash\n#SBATCH -J mbx-{0}\n#SBATCH -p background\n#SBATCH -c 1\n\
#SBATCH --mem=8gb\n#SBATCH -o mbx{0}-%j.out\n#SBATCH -e mbx{0}-%j.err\n\nchmod a+x ../Extract_nuisance_SF.py\n\n\
python ../Extract_nuisance_SF.py {1} {2} {3:1.5f} {4}\n\
\nwait\n\nexit 0".format(idx, alldb.field[i], alldb.id[i], alldb.z_grizli[i], alldb.lmass[i])]
        np.savetxt('../cluster_scripts/MBext_{0}.slrm'.format(idx), slrm_text, fmt='%s')    
        idx +=1 

In [6]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J mbx-385
#SBATCH -p background
#SBATCH -c 1
#SBATCH --mem=8gb
#SBATCH -o mbx385-%j.out
#SBATCH -e mbx385-%j.err

chmod a+x ../Extract_nuisance.py

python ../Extract_nuisance.py GSD 49362 1.19791

wait

exit 0
